In [ ]:
#!pip install selenium
#!pip install pandas

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By

path_chromedriver='C:/Projetos/env/chromedriver/chromedriver.exe'
s = Service(path_chromedriver)

driver = webdriver.Chrome(service=s)

In [ ]:
url = "https://www2.tjal.jus.br/cposg5/open.do"
driver.get(url)

# Login

1. Plugue seu certificado digital

2. Clique em Identificar-se

3. Digite sua senha

# Dados de Exemplo
A planilha abaixo está disponível no site do tribunal na URL abaixo

https://www.tjal.jus.br/arquivos_precatorios/a2094b214686d56cbe61639beaa40192.pdf

In [ ]:
import re
import pandas as pd
df = pd.read_csv("precatorios_al.csv", sep=",")
df

In [ ]:
def extrair_partes(elementoHTML):
    
    texto = ''
    
    for tr in driver.find_elements(By.CLASS_NAME, 'poloAtivo'):
        td = tr.find_elements(By.TAG_NAME, 'td')
        if re.match("Credor\w{0,2}\:", td[0].text): 
            texto = td[1].text
            break
    
    partes = texto.split("\n")
    autor = []
    advogados = []
    
    for parte in partes:
        if re.match("[\w\. ]*Advogad\w{1,2}\:", parte):
            advogados.append(parte.split(":")[1].strip())
        else:
            autor.append(parte)
    
    return autor, advogados

def consultar_processo(processo: str):
    driver.get("https://www2.tjal.jus.br/cposg5/open.do?gateway=true")
    
    driver.find_element(By.ID, 'numeroDigitoAnoUnificado').clear()
    driver.find_element(By.ID, 'foroNumeroUnificado').clear()
    
    driver.find_element(By.ID, 'numeroDigitoAnoUnificado').send_keys(processo[:15])
    driver.find_element(By.ID, 'foroNumeroUnificado').send_keys(processo[-4:])
    driver.find_element(By.ID, 'pbConsultar').click()
    
    if len(driver.find_elements(By.ID, 'mensagemRetorno')) == 0:
        return extrair_partes(driver.find_element(By.ID, 'tablePartesPrincipais'))
    else:
        return [], []
    
# Teste da função
processo = '0500437-46.2019.8.02.9003'

tupla = consultar_processo(processo)
tupla

# Iniciando o webscrapper
Ao rodar a célula abaixo, o selenium vai buscar os autores e os advogados do processo.
Após concluir a busca, a célula seguind incorpora os dados encontrados ao dataframe.

In [ ]:
Autor = []
Advogados = []
for index, row in df.iterrows():
    resultado = consultar_processo(row['Numero'])
    Autor.append(resultado[0])
    Advogados.append(resultado[1])

In [193]:
df['Autor'] = Autor
df['Advogados'] = Advogados
df

,Ordem,Numero,Data,Natureza,Orcamento,Origem,Valor,Atualizacao,Situacao,Autor,Advogados
0,490,0500657-73.2021.8.02.9003,16/6/2021,Comum,2022,TJAL,"25.019.669,39",jul./21,NaN,[Fundação Hospital da Agro-indústria do Açúcar...,"[Erivaldo Cavalcante Júnior, Henrique Carvalho..."
1,495,0500851-73.2021.8.02.9003,23/6/2021,Comum,2022,TJAL,"17.155.297,20",jul./21,NaN,[CITEL - Comércio Indústria e Técnicas Engenha...,"[Tarso & Moura Advogados, Acioli Araújo Cajuei..."
2,202,0500437-46.2019.8.02.9003,19/6/2019,Comum,2020,TJAL,"12.689.148,27",30/6/2020,NaN,[Ceres Machado Vasconcelos],[Cabral & Méro - Advogados Associados]
3,203,0500438-31.2019.8.02.9003,19/6/2019,Comum,2020,TJAL,"12.689.148,26",30/6/2020,NaN,[Álvaro José do Monte Vasconcelos],[Cabral & Méro - Advogados Associados]
4,20,0500126-06.2016.8.02.0000,8/3/2016,Comum,2017,TJAL,"7.078.511,14",31/3/2021,PAGO,[Clínica de Fraturas e Reabilitação de Maceió ...,"[Luiz Guilherme de Melo Lopes, Marcelo Henriqu..."
...,...,...,...,...,...,...,...,...,...,...,...
490,66,0500014-03.2017.8.02.0000,4/1/2017,Alimentar,2018,TJAL,"11.342,02",30/6/2020,Em via de Caucionamento de valor p Pgt.,[Sebastião Costa Pereira Sobrinho],[Alexsandre Victor Leite Peixoto]
491,67,0500015-85.2017.8.02.0000,4/1/2017,Alimentar,2018,TJAL,"10.437,17",30/6/2020,Em via de Caucionamento de valor p Pgt.,[],[]
492,424,0500519-09.2021.8.02.9003,01/062021,Alimentar,2022,TJAL,"6.513,20",jul./21,NaN,[João Carlos Glasherster da Rocha],[Nayara Cristina Magalhães de Barros]
493,32,0500354-78.2016.8.02.0000,2/8/2016,Alimentar,2018,TJAL,"4.906,68",31/3/2021,PAGO,[José Antônio Cavalcante Cerqueira],"[Clênio Pacheco Franco Júnior, Ayrton Alencar ..."


In [194]:
df.to_csv("resultados.csv", sep=";", encoding='utf-8-sig')